I had a post somewehre on this point. That programming z3 in python is staged metaprogramming.
The meta system python can do partial evaluation.




In [ ]:
def mypow(x:int,n:int) -> int:
    assert n >= 0
    if n == 0:
        return 1
    else:
        x * mypow(x,n-1)

# accumulator version?

# string version
# strings are a universal but somewhat structure free rep of code.
Code = str
def mypow2(n:int, x:Code) -> Code:
    if x == 0:
        return "1"
    else:
        f"{x} * {mypow2(x,n-1)}"

mypow = lambda x,n: 1 if n <= 0 else x * mypow(x,n-1)

mypow = Function("mypow", IntSort(), IntSort(), IntSort())
mypow_def = ForAll([x,n], mypow(n, x) == If(n <= 0, 1, x * mypow(n-1, x)))

# Partially evaled
def mypow(x:ExprRef, n:int) -> ExprRef:
    if n == 0:
        return IntVal(1)
    else:
        return x * mypow(x,n-1)




Pattern matching on z3 asts



I've ocmpained a bit before that it's crazy python doesn't have a good lambda library.
Well, it kind of does.


Lambda eval.
Z3 does do capture avoinding substitution. It can take care of de bruijn indices.
vs z3 simplify

Locally nameless matching. FreshConst + substitute makes this easy

Then we can use regular python pattern matching for lambda matching. Pretty cool!
I don't understand how the property mechanism works.



In [ ]:
z3.ExprRef.head = property(lambda self: self.decl().kind())
z3.ExprRef.args = property(lambda self: [self.arg(i) for i in range(self.num_args())])
z3.ExprRef.__match_args__ = ["head", "args"]
z3.QuantifierRef.open_term = property(lambda self: vars = FreshConst() (return vars, subst(self.body, []))) 
z3.QuantifierRef.__match_args__ = ["open_term"]

z3.QuantifierRef.__matmul__ = lambda self, other: z3.substitute(self.body, zip([z3.Var(n) for n in range(len(other)) , other]))

In [1]:
# It isn't all roses. Python's scoping rules are bonkers. It's intrinsically an imperative language.
match 42:
    case a:
        pass
print(a)

42


In [ ]:
def unify(a : AstRef, b : AstRef, vars):
    # or take in an existential equation.
    eqs = [(a,b)]
    subst = []
    sig = [] # miller unify?
    while eqs:
        a,b = eqs.pop()
        if a in vars:
            subst.append((a,b))
            map(lambda x: substitute(subst), eqs)
        elif b in vars:
            subst.append((b,a))
            map(lambda x: substitute(subst), eqs)
        


convert z3 ast to egglog for conditional simplification.